In [ ]:
import requests
# do a get request to the github api
r = requests.get('http://api.github.com/events')

# for standard output, print the status code of the response
r

In [ ]:
# show the content of the response
r.json()

In [ ]:
# in github api, we can see the version of the api changing the endpoint
r = requests.get('https://api.github.com/versions')

# parameter that shows the status code
r.status_code

In [ ]:
# show de version of the api
r.json()

In [ ]:
# show the url of the request
r.url

In [ ]:
# full url to request
api_base_url = 'https://api.github.com'
endpoint = 'users/amzn/repos'
url = f'{api_base_url}/{endpoint}'
url

In [ ]:
# to do more requests on the github API, a token is needed, it's provided in 'dev_settings' option from github site  
from my_config import PRIVATE_TOKEN
token = f'Bearer {PRIVATE_TOKEN}'
version = '2022-11-28'
headers = {}

headers['X-GitHub-Api-Version'] = version
headers['Authorization'] = token

r = requests.get(url, headers=headers)
r

In [ ]:
# how many repositories the amazon has in the first page
len(r.json())

In [ ]:
# request all pages from api

repos_list = []
repos_qtd = 0
last_page = False
page = 1

while (last_page == False):

    try:
        # try requesting
        url_page = f'{url}?page={page}'
        res = requests.get(url_page, headers=headers).json()

        # stop criteria
        if not res:
            last_page = True
        else:
            repos_list.append(res)
            page += 1

    except:
        last_page = True


# count how many repositories the api has
for i in range(0, len(repos_list)):
    repos_qtd += len(repos_list[i])

print(f'''
qtd de paginas: {len(repos_list)}
qtd de repositorios: {repos_qtd}''')

In [ ]:
repos_list[0][0]['name']

In [ ]:
# list of repositorie's names

repos_name = []

while True:
    for page in repos_list:
        for repo in page:
            repos_name.append(repo['name'])
    
    break
            
repos_name[:12]

In [ ]:
repos_list[0][1]['language']

In [ ]:
# list of repositorie's language uses

repos_language = []

for page in repos_list:
    for repo in page:
        repos_language.append(repo['language'])
    
            
repos_language[:9]

In [14]:
# library to manipulate and analyze data
import pandas as pd

# to create a DataFrame, each column is a key, and each key's values row are a list of rows
data = {'repo_names': repos_name, 'repos_language': repos_language}

df = pd.DataFrame(data)

# transforms a DataFrame into a dataset
df.to_csv('dataset_amazon_repos_name_and_language.csv')

In [ ]:
# endpoint to create a repository on github

api_base_url = 'https://api.github.com'


url = f'{api_base_url}/user/repos'
url

In [ ]:
# to create a repository a json parameter is needed, including 'name', 'description' and 'private' keys/values, these dictionary keys can be found in github docs

data = {
    'name': 'alura.python-e-apis-conhecendo-a-biblioteca-requests-file',
    'description': 'repositorio para armezenar: dataset_amazon_repos_name_and_language.csv',
    'private': False
}

# the method used to create a repository is 'post' (check de docs)
req = requests.post(url, headers=headers, json=data)


# after it being created, it can't be created again
req.status_code

In [17]:
# that python native library is used to encode and decode files into binary
import base64

path = 'dataset_amazon_repos_name_and_language.csv'

# some apis needs an enconded file because it is easier to transfer through the internet
with open(path, 'rb') as file:
    enconded_content = base64.b64encode(file.read())

In [ ]:
# endpoint to insert a content into a github repository

api_base_url = 'https://api.github.com'
owner = 'dihparaguai'
repo = 'alura.python-e-apis-conhecendo-a-biblioteca-requests-files'
path = 'dataset_amazon_repos_name_and_language.csv'

url = f'{api_base_url}/repos/{owner}/{repo}/contents/{path}'
url

In [ ]:
# to input a file into a repository, a json parameter is needed, including 'message' (commit) and 'content' keys/values, these dictionary keys can be found in github docs

# the content must be decoded using our charset 'utf-8'
data = {
    'message': 'adicionar dataset_amazon_repos_name_and_language.csv',
    'content': enconded_content.decode('utf-8')
}

# the method used to input a file into a repository is 'put' (check de docs)
req = requests.put(url, headers=headers, json=data)

# after it being created, it can't be created again
req.status_code